In [1]:
import pandas as pd
import json
import numpy as np

In [2]:
# --- DYNAMIC COLUMN MAPPING ---
# To use a different dataset, only update the column names on the right side of this dictionary.
# The internal names (the keys on the left) should not be changed.
COLUMN_MAPPINGS = {
    "primary": {
        "date": "Month",
        "grade": "Grade",
        "gauge": "Gauge",
        "national": "National",
        "weight_bought": "Weight Purchased",
        "weight_sold": "Weight Sold",
        "weight_consumed": "Weight Consumed",
    },
    "forecast_sold": {
        "date": "ds",
        "weight_sold": "yhat"
    },
    "forecast_consumed": {
        "date": "ds",
        "weight_consumed": "yhat"
    },
    "rigs": {
        "date": "Date_Month",
        "region": "Region",
        "count_raw": "Total Rig Count"
    },
    "oil": {
        "date": "Date_Month",
        "price_raw": "Oil_Price"
    }
}

In [7]:
# --- FILE PATHS ---
# Update these paths to point to your data files.
FILE_PATHS = {
    "primary": "../Data/processed/final_merge.csv",
    "rigs": "../Data/external/rigs_per_month.csv",
    "oil": "../Data/external/oil_prices_monthly.csv",
    "forecast_sold": "../Data/processed/Forecasts/forecast_sold.csv",
    "forecast_consumed": "../Data/processed/Forecasts/forecast_consumed.csv"
}

In [9]:
def create_final_dashboard():
    """
    Loads all data sources and generates a final dashboard with a National/International filter,
    forecasting, KPIs, and dynamic controls.
    """
    try:
        # --- 1. Load and Standardize Primary Data ---
        p_map = COLUMN_MAPPINGS['primary']
        primary_df = pd.read_csv(FILE_PATHS['primary'], usecols=p_map.values()).rename(columns={v: k for k, v in p_map.items()})
        primary_df['date'] = pd.to_datetime(primary_df['date'])
        primary_wide_json = primary_df.to_json(orient='records', date_format='iso')

        # --- 2. Reshape (Melt) the Data for Time Series Plotting ---
        id_vars = ['date', 'grade', 'gauge', 'national'] # <-- ADDED 'national'
        primary_long = pd.melt(primary_df, id_vars=id_vars,
                               value_vars=['weight_bought', 'weight_sold', 'weight_consumed'],
                               var_name='type_long', value_name='weight_raw')

        type_map = {'weight_bought': 'Purchase', 'weight_sold': 'Sale', 'weight_consumed': 'Consumption'}
        primary_long['type'] = primary_long['type_long'].map(type_map)
        primary_long.drop('type_long', axis=1, inplace=True)
        primary_long['gauge'] = primary_long['gauge'].astype(str)
        primary_long = primary_long.sort_values('date')

        # --- 3. Calculate MA and MoM% for Weight ---
        grouped = primary_long.groupby(['grade', 'gauge', 'type', 'national']) # <-- ADDED 'national'
        primary_long['weight_ma'] = grouped['weight_raw'].transform(lambda x: x.rolling(4, 1).mean())
        primary_long['weight_mom'] = grouped['weight_raw'].transform(lambda x: x.pct_change() * 100)

        primary_long.fillna(0, inplace=True)
        primary_long_json = primary_long.to_json(orient='records', date_format='iso')

        # --- 4. Load and Process Aggregate Forecast Data ---
        fs_map = COLUMN_MAPPINGS['forecast_sold']
        fc_map = COLUMN_MAPPINGS['forecast_consumed']
        forecast_sold_df = pd.read_csv(FILE_PATHS['forecast_sold'], usecols=fs_map.values()).rename(columns={v: k for k, v in fs_map.items()})
        forecast_consumed_df = pd.read_csv(FILE_PATHS['forecast_consumed'], usecols=fc_map.values()).rename(columns={v: k for k, v in fc_map.items()})

        forecast_df = pd.merge(forecast_sold_df, forecast_consumed_df, on='date', how='outer')
        forecast_df['date'] = pd.to_datetime(forecast_df['date'])

        forecast_long = pd.melt(forecast_df, id_vars=['date'], value_vars=['weight_sold', 'weight_consumed'],
                                var_name='type_long', value_name='weight_raw')
        forecast_long['type'] = forecast_long['type_long'].map(type_map)
        forecast_long.drop('type_long', axis=1, inplace=True)

        forecast_long = forecast_long.sort_values('date')
        forecast_long['weight_ma'] = forecast_long.groupby('type')['weight_raw'].transform(lambda x: x.rolling(4, 1).mean())
        forecast_long['weight_mom'] = forecast_long.groupby('type')['weight_raw'].transform(lambda x: x.pct_change() * 100).fillna(0)
        forecast_data_json = forecast_long.to_json(orient='records', date_format='iso')

        # --- 5. Load and Process Macro Data ---
        r_map, o_map = COLUMN_MAPPINGS['rigs'], COLUMN_MAPPINGS['oil']
        rigs_df = pd.read_csv(FILE_PATHS['rigs'], usecols=r_map.values()).rename(columns={v: k for k, v in r_map.items()})
        oil_df = pd.read_csv(FILE_PATHS['oil'], usecols=o_map.values()).rename(columns={v: k for k, v in o_map.items()})
        for df in [rigs_df, oil_df]: df['date'] = pd.to_datetime(df['date'])

        total_rigs = rigs_df.groupby('date')['count_raw'].sum().reset_index(); total_rigs['region'] = 'All'
        rigs_df = pd.concat([rigs_df, total_rigs], ignore_index=True)
        rigs_df['count_ma'] = rigs_df.groupby('region')['count_raw'].transform(lambda x: x.rolling(4, 1).mean())
        rigs_df['count_mom'] = rigs_df.groupby('region')['count_raw'].pct_change().fillna(0) * 100
        rig_data_json = rigs_df.to_json(orient='records', date_format='iso')

        oil_df['price_ma'] = oil_df['price_raw'].rolling(4, 1).mean()
        oil_df['price_mom'] = oil_df['price_raw'].pct_change().fillna(0) * 100
        oil_data_json = oil_df.to_json(orient='records', date_format='iso')

    except (FileNotFoundError, KeyError) as e:
        print(f"Error: {e}. Check FILE_PATHS and COLUMN_MAPPINGS in the script.")
        return

    # --- 6. Prepare Data for HTML/JS ---
    grade_to_gauge_map = primary_long.groupby('grade')['gauge'].unique().apply(lambda x: sorted(list(x))).to_dict()
    grade_gauge_map_json = json.dumps(grade_to_gauge_map)
    all_grades = sorted(primary_long['grade'].unique().tolist())
    all_gauges = sorted(primary_long['gauge'].unique().tolist())
    regions = ['All'] + sorted(rigs_df[rigs_df['region'] != 'All']['region'].unique().tolist())
    region_options = "".join([f'<option value="{r}">{r}</option>' for r in regions])
    min_date = primary_long['date'].min().strftime('%Y-%m-%d')
    max_date = forecast_long['date'].max().strftime('%Y-%m-%d') if not forecast_long.empty else primary_long['date'].max().strftime('%Y-%m-%d')

    # --- 7. HTML Template ---
    html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Coil Tubing Dashboard with Forecasting</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <style>
        body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, sans-serif; margin: 0; padding: 0; background-color: #f4f4f9; color: #333; }}
        .dashboard-container {{ display: grid; grid-template-columns: 320px 1fr; grid-template-rows: auto 1fr; height: 100vh; gap: 15px; padding: 15px; box-sizing: border-box; }}
        .header {{ grid-column: 1 / -1; padding: 15px; background-color: #fff; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); text-align: center; }}
        .header h1 {{ margin: 0; color: #2c3e50; font-size: 1.8em; }}
        .controls-panel {{ grid-row: 2 / 3; background-color: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); overflow-y: auto; }}
        .chart-container {{ grid-row: 2 / 3; grid-column: 2 / 3; background-color: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); display: flex; align-items: center; justify-content: center; }}
        .control-group {{ margin-bottom: 20px; }}
        .control-group label, .control-group h3 {{ display: block; margin-bottom: 8px; font-weight: 600; color: #34495e; }}
        .control-group h3 {{ font-size: 1.1em; margin-top: 25px; border-bottom: 1px solid #eee; padding-bottom: 5px; }}
        input[type="date"], select {{ width: 100%; padding: 8px; border-radius: 4px; border: 1px solid #ccc; box-sizing: border-box; }}
        .radio-group label, .checkbox-group label {{ display: inline-block; margin-right: 10px; font-weight: normal; cursor: pointer; }}
        label[disabled="disabled"] {{ color: #ccc; cursor: not-allowed; }}
        .multiselect {{ position: relative; }}
        .select-box {{ border: 1px solid #ccc; border-radius: 4px; padding: 8px; cursor: pointer; display: flex; justify-content: space-between; align-items: center; }}
        .select-box::after {{ content: '▼'; font-size: 0.8em; }}
        .options-container {{ display: none; position: absolute; background: white; border: 1px solid #ccc; border-radius: 4px; max-height: 200px; overflow-y: auto; width: 100%; z-index: 10; box-shadow: 0 4px 8px rgba(0,0,0,0.1); }}
        .options-container label {{ display: block; padding: 8px 12px; cursor: pointer; }}
        .options-container label:hover {{ background-color: #f0f0f0; }}
        .options-container.visible {{ display: block; }}
        .selected-badge {{ background-color: #3498db; color: white; padding: 2px 6px; border-radius: 10px; font-size: 0.8em; margin-left: 5px; }}
        .kpi-container {{ display: grid; grid-template-columns: 1fr 1fr; gap: 10px; margin-top: 15px; }}
        .kpi-card {{ background-color: #f8f9fa; border: 1px solid #dee2e6; border-radius: 6px; padding: 10px; text-align: center; }}
        .kpi-card h2 {{ margin: 0 0 5px 0; font-size: 1.8em; color: #2c3e50; }}
        .kpi-card p {{ margin: 0; font-size: 0.8em; color: #6c757d; }}
        #calc-kpi-btn {{ width: 100%; padding: 10px; background-color: #28a745; color: white; border: none; border-radius: 4px; font-size: 1em; cursor: pointer; }}
        #calc-kpi-btn:hover {{ background-color: #218838; }}
    </style>
</head>
<body>
    <div class="dashboard-container">
        <div class="header"><h1>Coil Tubing & Macro Dashboard</h1></div>
        <div class="controls-panel">
            <h3>Time Series Controls</h3>
            <div class="control-group checkbox-group">
                <label>Data View</label>
                <input type="checkbox" id="view-purchase" name="view-type" value="Purchase" checked onchange="updateChart()"><label for="view-purchase">Purchases</label>
                <input type="checkbox" id="view-sale" name="view-type" value="Sale" checked onchange="updateChart()"><label for="view-sale">Sales</label>
                <input type="checkbox" id="view-consumption" name="view-type" value="Consumption" onchange="updateChart()"><label for="view-consumption">Consumption</label>
            </div>
            <div class="control-group">
                <label>Order Type</label>
                <div class="radio-group">
                    <input type="radio" id="nat-both" name="national-type" value="Both" checked onchange="updateChart()"><label for="nat-both">Both</label>
                    <input type="radio" id="nat-national" name="national-type" value="National" onchange="updateChart()"><label for="nat-national">National</label>
                    <input type="radio" id="nat-international" name="national-type" value="International" onchange="updateChart()"><label for="nat-international">Int'l</label>
                </div>
            </div>
            <div class="control-group">
                <label>Grade</label>
                <div id="grade-multiselect" class="multiselect"><div class="select-box" onclick="toggleOptions('grade-options')"></div><div id="grade-options" class="options-container"></div></div>
            </div>
            <div class="control-group">
                <label>Gauge</label>
                <div id="gauge-multiselect" class="multiselect"><div class="select-box" onclick="toggleOptions('gauge-options')"></div><div id="gauge-options" class="options-container"></div></div>
            </div>
            <div class="control-group">
                <label>Display As (for multi-selection)</label>
                <div class="radio-group">
                    <input type="radio" id="display-sum" name="display-type" value="sum" checked onchange="updateChart()"><label for="display-sum">Summed</label>
                    <input type="radio" id="display-separate" name="display-type" value="separate" onchange="updateChart()"><label for="display-separate">Separate</label>
                </div>
            </div>
             <div class="control-group checkbox-group">
                <input type="checkbox" id="show-forecast" onchange="updateChart()"><label id="show-forecast-label" for="show-forecast">Show Forecast</label>
            </div>
            <div class="control-group">
                <label>Date Range</label>
                <input type="date" id="start-date" value="{min_date}" onchange="updateChart()"><input type="date" id="end-date" value="{max_date}" onchange="updateChart()" style="margin-top: 5px;">
            </div>
            <div class="control-group">
                <label>Value Type</label>
                <div class="radio-group">
                    <input type="radio" id="valuetype-raw" name="valuetype" value="raw" checked onchange="updateChart()"><label for="valuetype-raw">Raw</label>
                    <input type="radio" id="valuetype-ma" name="valuetype" value="ma" onchange="updateChart()"><label for="valuetype-ma">MA</label>
                    <input type="radio" id="valuetype-mom" name="valuetype" value="mom" onchange="updateChart()"><label for="valuetype-mom">MoM%</label>
                </div>
            </div>
            <h3>Summary Metrics (Historical)</h3>
             <div class="control-group">
                 <button id="calc-kpi-btn" onclick="calculateMetrics()">Calculate Summary Metrics</button>
                 <div class="kpi-container">
                     <div class="kpi-card"><h2 id="kpi-abs-gap">-</h2><p>Absolute Gap (Tons)</p></div>
                     <div class="kpi-card"><h2 id="kpi-pct-gap">-</h2><p>% Gap (vs. Bought)</p></div>
                     <div class="kpi-card"><h2 id="kpi-sell-thru">-</h2><p>Sell-Through Rate</p></div>
                     <div class="kpi-card"><h2 id="kpi-cons-cov">-</h2><p>Consumption Coverage</p></div>
                 </div>
             </div>
            <h3>Macro Data Overlays</h3>
            <div class="control-group checkbox-group">
                <input type="checkbox" id="show-rigs" onchange="toggleRigControls(); updateChart();"><label for="show-rigs">Rig Count</label>
                <input type="checkbox" id="show-oil" onchange="updateChart()"><label for="show-oil">Oil Price</label>
            </div>
            <div class="control-group" id="rig-controls" style="display:none;">
                <label for="region-select">Region</label>
                <select id="region-select" onchange="updateChart()">{region_options}</select>
            </div>
            <div class="control-group">
                <label>Macro Value Type</label>
                <div class="radio-group">
                    <input type="radio" id="macro-valuetype-raw" name="macro-valuetype" value="raw" checked onchange="updateChart()"><label for="macro-valuetype-raw">Raw</label>
                    <input type="radio" id="macro-valuetype-ma" name="macro-valuetype" value="ma" onchange="updateChart()"><label for="macro-valuetype-ma">MA</label>
                    <input type="radio" id="macro-valuetype-mom" name="macro-valuetype" value="mom" onchange="updateChart()"><label for="macro-valuetype-mom">MoM%</label>
                </div>
            </div>
        </div>
        <div class="chart-container"><div id="plotly-chart" style="width:100%;height:100%;"></div></div>
    </div>

    <script id="primary-data-long-json" type="application/json">{primary_long_json}</script>
    <script id="primary-data-wide-json" type="application/json">{primary_wide_json}</script>
    <script id="forecast-data-json" type="application/json">{forecast_data_json}</script>
    <script id="rig-data-json" type="application/json">{rig_data_json}</script>
    <script id="oil-data-json" type="application/json">{oil_data_json}</script>
    <script id="grade-gauge-map-json" type="application/json">{grade_gauge_map_json}</script>

    <script>
        const primaryDataLong = JSON.parse(document.getElementById('primary-data-long-json').textContent);
        const primaryDataWide = JSON.parse(document.getElementById('primary-data-wide-json').textContent);
        const forecastData = JSON.parse(document.getElementById('forecast-data-json').textContent);
        const rigData = JSON.parse(document.getElementById('rig-data-json').textContent);
        const oilData = JSON.parse(document.getElementById('oil-data-json').textContent);
        const gradeToGaugeMap = JSON.parse(document.getElementById('grade-gauge-map-json').textContent);
        const ALL_GRADES = {json.dumps(all_grades)};
        const ALL_GAUGES = {json.dumps(all_gauges)};
        const PLOTLY_COLORS = Plotly.d3.scale.category10().range();
        const TYPE_STYLES = {{'Purchase': {{ color: 'royalblue' }}, 'Sale': {{ color: 'firebrick' }}, 'Consumption': {{ color: 'seagreen' }} }};

        function toggleOptions(containerId) {{ document.getElementById(containerId).classList.toggle('visible'); }}

        function getMultiSelectValues(containerId) {{
            const values = [];
            const allCheckbox = document.querySelector(`#${{containerId}} input[value="All"]`);
            if (allCheckbox && allCheckbox.checked) return ['All'];
            document.querySelectorAll(`#${{containerId}} input:checked`).forEach(cb => {{ if (cb.value !== 'All') values.push(cb.value); }});
            return values.length > 0 ? values : ['All'];
        }}

        function getCheckboxValues(name) {{
            const values = [];
            document.querySelectorAll(`input[name="${{name}}"]:checked`).forEach(cb => values.push(cb.value));
            return values;
        }}

        function updateSelectBoxText(containerId, values, itemType) {{
            const selectBox = document.querySelector(`#${{containerId}} .select-box`);
            const allOptionsCount = (itemType === 'grade' ? ALL_GRADES.length : ALL_GAUGES.length);
            if (values[0] === 'All' || values.length === 0 || values.length >= allOptionsCount) {{
                selectBox.innerHTML = `All ${{itemType}}s`;
            }} else if (values.length === 1) {{
                selectBox.innerHTML = values[0];
            }} else {{
                selectBox.innerHTML = `Multiple <span class="selected-badge">${{values.length}}</span>`;
            }}
        }}

        function populateMultiSelect(containerId, options, isGrade = false) {{
            const container = document.getElementById(containerId);
            const onchange_function = isGrade ? "updateGaugeOptions()" : "updateChart()";
            const all_onchange_handler = `handleAllCheckbox(this, '${{containerId}}', ${{isGrade}})`;
            let html = `<label><input type="checkbox" value="All" onchange="${{all_onchange_handler}}"> All</label>`;
            options.forEach(opt => {{ html += `<label><input type="checkbox" value="${{opt}}" onchange="${{onchange_function}}"> ${{opt}}</label>`; }});
            container.innerHTML = html;
        }}

        function handleAllCheckbox(allCheckbox, containerId, isGrade) {{
            document.querySelectorAll(`#${{containerId}} input`).forEach(cb => cb.checked = allCheckbox.checked);
            if (isGrade) updateGaugeOptions(); else updateChart();
        }}

        function updateGaugeOptions() {{
            const selectedGrades = getMultiSelectValues('grade-options');
            let availableGauges = new Set();
            if (selectedGrades[0] === 'All') {{
                ALL_GAUGES.forEach(g => availableGauges.add(g));
            }} else {{
                selectedGrades.forEach(grade => {{
                    if (gradeToGaugeMap[grade]) gradeToGaugeMap[grade].forEach(g => availableGauges.add(g));
                }});
            }}
            populateMultiSelect('gauge-options', Array.from(availableGauges).sort(), false);
            updateChart();
        }}

        function checkForecastAvailability() {{
            const selectedGrades = getMultiSelectValues('grade-options');
            const selectedGauges = getMultiSelectValues('gauge-options');
            const forecastCheckbox = document.getElementById('show-forecast');
            const forecastLabel = document.getElementById('show-forecast-label');
            if (selectedGrades[0] === 'All' && selectedGauges[0] === 'All') {{
                forecastCheckbox.disabled = false;
                forecastLabel.removeAttribute('disabled');
            }} else {{
                forecastCheckbox.disabled = true;
                forecastCheckbox.checked = false;
                forecastLabel.setAttribute('disabled', 'disabled');
            }}
        }}

        function groupBy(data, keys, metric) {{
            const result = {{}};
            data.forEach(item => {{
                const key = keys.map(k => item[k]).join('|');
                if (!result[key]) result[key] = {{ ...item, [metric]: 0 }};
                if (typeof item[metric] === 'number') result[key][metric] += item[metric];
            }});
            return Object.values(result).sort((a, b) => new Date(a.date) - new Date(b.date));
        }}

        function updateChart() {{
            checkForecastAvailability();
            updateSelectBoxText('grade-multiselect', getMultiSelectValues('grade-options'), 'grade');
            updateSelectBoxText('gauge-multiselect', getMultiSelectValues('gauge-options'), 'gauge');

            const selectedGrades = getMultiSelectValues('grade-options');
            const selectedGauges = getMultiSelectValues('gauge-options');
            const nationalType = document.querySelector('input[name="national-type"]:checked').value;
            const startDate = document.getElementById('start-date').value;
            const endDate = document.getElementById('end-date').value;
            const primaryValueType = document.querySelector('input[name="valuetype"]:checked').value;
            const displayType = document.querySelector('input[name="display-type"]:checked').value;
            const dataViews = getCheckboxValues('view-type');
            const showForecast = document.getElementById('show-forecast').checked;

            const filteredPrimary = primaryDataLong.filter(row =>
                (selectedGrades[0] === 'All' || selectedGrades.includes(row.grade)) &&
                (selectedGauges[0] === 'All' || selectedGauges.includes(row.gauge)) &&
                (dataViews.includes(row.type)) &&
                (nationalType === 'Both' || (nationalType === 'National' && row.national == 1) || (nationalType === 'International' && row.national == 0)) &&
                (row.date.substring(0, 10) >= startDate && row.date.substring(0, 10) <= endDate)
            );

            const yCol = `weight_${{primaryValueType}}`;
            const yAxisTitle = `Weight (${{primaryValueType.toUpperCase()}})`;
            let chartTitle = `Monthly Weight`;

            let allTraces = [];
            const keyToSplitBy = (selectedGrades.length > 1 && selectedGrades[0] !== 'All') ? 'grade' : 'gauge';
            const itemsToShow = (keyToSplitBy === 'grade') ? selectedGrades : selectedGauges;
            const shouldSplit = displayType === 'separate' && !(itemsToShow.length <= 1 || itemsToShow[0] === 'All');

            if (!shouldSplit) {{
                dataViews.forEach(view => {{
                    const sumData = groupBy(filteredPrimary.filter(d => d.type === view), ['date'], yCol);
                    allTraces.push({{ x: sumData.map(d=>d.date), y: sumData.map(d=>d[yCol]), name: `${{view}}`, legendgroup: view, mode: 'lines+markers', line: TYPE_STYLES[view] }});

                    if (showForecast && view !== 'Purchase') {{
                        const forecastViewData = forecastData.filter(d => d.type === view && d.date.substring(0,10) >= startDate && d.date.substring(0,10) <= endDate);
                        if (forecastViewData.length > 0) {{
                            const lastHistoricalPoint = sumData.length > 0 ? sumData[sumData.length - 1] : null;
                            const forecastX = forecastViewData.map(d => d.date);
                            const forecastY = forecastViewData.map(d => d[yCol]);
                            if (lastHistoricalPoint) {{
                                forecastX.unshift(lastHistoricalPoint.date);
                                forecastY.unshift(lastHistoricalPoint[yCol]);
                            }}
                            allTraces.push({{ x: forecastX, y: forecastY, name: `${{view}} (F)`, legendgroup: view, showlegend: false, mode: 'lines', line: {{...TYPE_STYLES[view], dash: 'dash'}} }});
                        }}
                    }}
                }});
            }} else {{
                chartTitle += ` by ${{keyToSplitBy}}`;
                itemsToShow.forEach((item, i) => {{
                    const itemData = filteredPrimary.filter(d => d[keyToSplitBy] === item);
                    const color = PLOTLY_COLORS[i % PLOTLY_COLORS.length];
                    dataViews.forEach(view => {{
                        const viewData = groupBy(itemData.filter(d => d.type === view), ['date'], yCol);
                        allTraces.push({{ x: viewData.map(d=>d.date), y: viewData.map(d=>d[yCol]), name: `${{view.slice(0,1)}}. ${{item}}`, mode: 'lines+markers', line:{{color: color}} }});
                    }});
                }});
            }}

            const layout = {{ title: chartTitle, xaxis: {{ title: 'Date' }}, yaxis: {{ title: yAxisTitle }}, margin: {{ l: 80, r: 80, b: 50, t: 50 }}, legend: {{ orientation: 'h', yanchor: 'bottom', y: 1.02, xanchor: 'right', x: 1 }} }};
            addMacroTraces(allTraces, layout);
            Plotly.newPlot('plotly-chart', allTraces, layout, {{responsive: true}});
        }}

        function calculateMetrics() {{
            const selectedGrades = getMultiSelectValues('grade-options');
            const selectedGauges = getMultiSelectValues('gauge-options');
            const nationalType = document.querySelector('input[name="national-type"]:checked').value;
            const startDate = document.getElementById('start-date').value;
            const endDate = document.getElementById('end-date').value;

            const forecastStartDate = '2025-02-01';

            const filteredWide = primaryDataWide.filter(row =>
                (selectedGrades[0] === 'All' || selectedGrades.includes(row.grade)) &&
                (selectedGauges[0] === 'All' || selectedGauges.includes(row.gauge)) &&
                (nationalType === 'Both' || (nationalType === 'National' && row.national == 1) || (nationalType === 'International' && row.national == 0)) &&
                (row.date.substring(0, 10) >= startDate && row.date.substring(0, 10) <= endDate) &&
                (row.date.substring(0, 10) < forecastStartDate) // Only historical data for KPIs
            );

            if (filteredWide.length === 0) {{
                ['abs-gap', 'pct-gap', 'sell-thru', 'cons-cov'].forEach(id => document.getElementById(`kpi-${{id}}`).innerText = 'N/A');
                return;
            }}

            const totals = filteredWide.reduce((acc, row) => {{
                acc.wb += row.weight_bought || 0; acc.ws += row.weight_sold || 0; acc.wc += row.weight_consumed || 0;
                return acc;
            }}, {{ wb: 0, ws: 0, wc: 0 }});

            const abs_gap = totals.wb - totals.ws;
            const pct_gap = totals.wb > 0 ? ((totals.wb - totals.ws) / totals.wb) * 100 : 0;
            const sell_thru = totals.wb > 0 ? (totals.ws / totals.wb) * 100 : 0;
            const cons_cov = totals.wb > 0 ? (totals.wc / totals.wb) * 100 : 0;

            document.getElementById('kpi-abs-gap').innerText = abs_gap.toLocaleString(undefined, {{maximumFractionDigits: 0}});
            document.getElementById('kpi-pct-gap').innerText = `${{pct_gap.toFixed(1)}}%`;
            document.getElementById('kpi-sell-thru').innerText = `${{sell_thru.toFixed(1)}}%`;
            document.getElementById('kpi-cons-cov').innerText = `${{cons_cov.toFixed(1)}}%`;
        }}

        function addMacroTraces(traces, layout) {{
            const macroValueType = document.querySelector('input[name="macro-valuetype"]:checked').value;
            const startDate = document.getElementById('start-date').value;
            const endDate = document.getElementById('end-date').value;
            let secondaryAxisTitles = [];
            if (document.getElementById('show-rigs').checked) {{
                const selectedRegion = document.getElementById('region-select').value;
                const yCol = `count_${{macroValueType}}`;
                const yName = `Rig Count (${{macroValueType.toUpperCase()}})`;
                const filtered = rigData.filter(r => r.region === selectedRegion && r.date.substring(0,10) >= startDate && r.date.substring(0,10) <= endDate);
                traces.push({{ x: filtered.map(d=>d.date), y: filtered.map(d=>d[yCol]), name: `${{selectedRegion}} Rigs`, mode:'lines', line:{{color:'darkcyan'}}, yaxis:'y2' }});
                secondaryAxisTitles.push(yName);
            }}
            if (document.getElementById('show-oil').checked) {{
                const yCol = `price_${{macroValueType}}`;
                const yName = `Oil Price (${{macroValueType.toUpperCase()}})`;
                const filtered = oilData.filter(r => r.date.substring(0,10) >= startDate && r.date.substring(0,10) <= endDate);
                traces.push({{ x: filtered.map(d=>d.date), y: filtered.map(d=>d[yCol]), name: yName, mode:'lines', line:{{color:'goldenrod'}}, yaxis:'y2' }});
                secondaryAxisTitles.push(yName);
            }}
            if (secondaryAxisTitles.length > 0) {{
                layout.yaxis2 = {{ title: secondaryAxisTitles.join(' / '), overlaying: 'y', side: 'right', showgrid: false, zeroline: false }};
            }}
        }}

        function toggleRigControls() {{ document.getElementById('rig-controls').style.display = document.getElementById('show-rigs').checked ? 'block' : 'none'; }}

        document.addEventListener('DOMContentLoaded', () => {{
            populateMultiSelect('grade-options', ALL_GRADES, true);
            populateMultiSelect('gauge-options', ALL_GAUGES, false);
            updateChart();
            window.addEventListener('click', e => {{
                document.querySelectorAll('.multiselect .options-container.visible').forEach(container => {{
                    if (!container.parentElement.contains(e.target)) container.classList.remove('visible');
                }});
            }});
        }});
    </script>
</body>
</html>
"""
    try:
        with open('../Data/processed/Dashboards/dashboard_country_split.html', 'w', encoding='utf-8') as f:
            f.write(html_template)
        print("Successfully created dashboard_final.html")
    except IOError as e:
        print(f"Error writing to file: {e}")

In [10]:
create_final_dashboard()

Successfully created dashboard_final.html


C:\Users\Amine Bakkoury\AppData\Local\Temp\ipykernel_26988\2678537174.py:65: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  oil_df['price_mom'] = oil_df['price_raw'].pct_change().fillna(0) * 100
